In [1]:
def get_enquiry_responce_from_server(rid):
    enquiry_limit = 30
    current_enquiry_count = 0
    payment_request_obj = PaymentRequest.objects.get(rid=rid)
    if EnquiryRequest.objects.filter(rid=payment_request_obj.rid).exists():
        current_enquiry_count = EnquiryRequest.objects.filter(rid=payment_request_obj.rid).count()
    if current_enquiry_count <= enquiry_limit:
        is_response_available = False
        brn = None
        if PaymentRequestResponse.objects.filter(payment_request_id=payment_request_obj.id).exists():
            payment_responce_obj = PaymentRequestResponse.objects.get(payment_request_id=payment_request_obj.id)
            brn = payment_responce_obj.brn
            is_response_available = True
        # dict for filling white space in ACII method
        pad_text_dict = {
          0: '\x00', 1: '\x01', 2: '\x02', 3: '\x03', 4: '\x04', 5: '\x05', 6: '\x06', 7: '\x07', 8: '\x08', 9: '\x09', 10: '\x0a', 11: '\x0b', 12: '\x0c', 13: '\x0d', 14: '\x0e', 15: '\x0f', 16: '\x10'
        }
        ver = '1.0' #version
        cid = 5648 # cid for aavin given by axis bank
        typ = 'PRD' # test environment
        rid = payment_request_obj.rid # random reference number
        crn = payment_request_obj.crn # customer reference number
        key_before_sha_encrypt = '{}{}{}{}'.format(cid,rid,crn,checksum_key)
        cks = hashlib.sha256(key_before_sha_encrypt.encode()).hexdigest()
        if is_response_available:
            final_string = 'CID={}&RID={}&CRN={}&VER={}&TYP={}&BRN={}&CKS={}'.format(cid,rid,crn,ver,typ,brn,cks)
        else:
            final_string = 'CID={}&RID={}&CRN={}&VER={}&TYP={}&CKS={}'.format(cid,rid,crn,ver,typ,cks)
        obj = AES.new(encryption_key, AES.MODE_ECB, 'OPENSSL_RAW_DATA')
        pad_count = (16 - len(final_string) % 16)
        padded_text = final_string + (16 - len(final_string) % 16) * pad_text_dict[pad_count]
        cipertext = obj.encrypt(padded_text)
        cipertext = b64encode(cipertext).decode('utf-8')
        enquiry_request_obj = EnquiryRequest(rid=payment_request_obj.rid,
                                          status_id=1, # requested
                                          encrypted_string=cipertext,
                                          decrypted_string=final_string,
                                          ver=ver,
                                          cid=cid,
                                          typ=typ,
                                          crn=crn,
                                          cks=cks,
                                          enquiry_make_by_id=1)
        if is_response_available:
            enquiry_request_obj.brn=brn
        enquiry_request_obj.save()
        response_return = requests.post('https://easypay.axisbank.co.in/index.php/api/enquiry', {'i': cipertext})
        encrypted_key_from_enquiry = response_return.text
        print(encrypted_key_from_enquiry)
        # encrypted_key_from_enquiry = 'By9ehZvsAZycxvvPI8Qy9vhePd+HmEf87QizvjiB8MT9NkQhBB4VFT8v9Kqff1nH4VYVowx+2FyjKprlFHA/3zGt0HF5BjVq79cFiw8LCq30a92nJaiHnar0AGBznJwufm9S7no1zYsjCQZhzaywaZfgILKCqzXW7B6hWw/eNm7DL3KSZJs74RGCUAwAzUXq8aYt7e+k2OVNJbtK+cFHSt4FdRBCwexIv9ujqEPa7c+VPYYScG3kdDkB/mTcGye7yObrn/oof5PnhBH4rD8W7koFUb0SBaU25Wz50XsoTdE='

        encrypted_key_from_enquiry = b64decode(encrypted_key_from_enquiry)
        decipher = AES.new(encryption_key, AES.MODE_ECB)
        decrypted_text = decipher.decrypt(encrypted_key_from_enquiry).decode("utf-8")
        print(decrypted_text)
        # split plain text by &
        splited_string = decrypted_text.split('&')

        # status code with ids
        transaction_status_obj = PaymentTransactionStatus.objects.all()
        transaction_status_list = list(transaction_status_obj.values_list('id', 'name', 'code'))
        transaction_status_column = ['id', 'name', 'code']
        transaction_status_df = pd.DataFrame(transaction_status_list, columns=transaction_status_column)
        transaction_status_dict = transaction_status_df.groupby('code').apply(lambda x: x.to_dict('r')[0]).to_dict()

        # create dict for key and respected value
        decrypted_string_and_value = {
          'BRN': None,
          'STC': None,
          'RMK': None,
          'TRN': None,
          'TET': None,
          'PMD': None,
          'RID': None,
          'VER': None,
          'CID': None,
          'TYP': None,
          'CRN': None,
          'CNY': None,
          'AMT': None,
          'CKS': None,    
        }

        # store the parameters in dict for update payemnt responce table
        for string in splited_string:
            temp_split = string.split('=')
            decrypted_string_and_value[temp_split[0]] = temp_split[1]

        # # create enquiry responce obj
        # transaction_date_time_in_format = transaction_date_time_in_format.astimezone(indian)
        if decrypted_string_and_value['TRN'] == '':
            decrypted_string_and_value['TRN'] = decrypted_string_and_value['RID'] + '_cancelled'
        if decrypted_string_and_value['TET'] == '':
            transaction_date_time_in_format = datetime.datetime.now()
        else:
            transaction_date_time_in_format = datetime.datetime.strptime(decrypted_string_and_value['TET'], '%Y/%m/%d %H:%M:%S %p')
        if decrypted_string_and_value['AMT'] == '':
            decrypted_string_and_value['AMT'] = 0
        if decrypted_string_and_value['PMD'] == '':
            decrypted_string_and_value['PMD'] = ' '
        if decrypted_string_and_value['CNY'] == '':
            decrypted_string_and_value['CNY'] = ' '
        enquiry_responce_obj = EnquiryRequestResponse(enquiry_request_id=enquiry_request_obj.id,
                                                  rid=decrypted_string_and_value['RID'],
                                                  status_id=transaction_status_dict[decrypted_string_and_value['STC']]['id'],
                                                  encrypted_string=str(encrypted_key_from_enquiry),
                                                  decrypted_string=decrypted_text,
                                                  brn=decrypted_string_and_value['BRN'],
                                                  trn=decrypted_string_and_value['TRN'],
                                                  tet=transaction_date_time_in_format,
                                                  pmd=decrypted_string_and_value['PMD'],
                                                  stc=decrypted_string_and_value['STC'],
                                                  rmk=decrypted_string_and_value['RMK'],
                                                  ver=decrypted_string_and_value['VER'],
                                                  cid=decrypted_string_and_value['CID'],
                                                  typ=decrypted_string_and_value['TYP'],
                                                  crn=decrypted_string_and_value['CRN'],
                                                  cny=decrypted_string_and_value['CNY'],
                                                  amt=decrypted_string_and_value['AMT'],
                                                  cks=decrypted_string_and_value['CKS'])

        enquiry_responce_obj.save()
        return decrypted_string_and_value['STC']
    else:
        return '111'

In [2]:
def send_amount_to_agent_wallet(agent_id, amount):
    agent_wallet_obj = AgentWallet.objects.get(agent_id=agent_id)
    current_balance = agent_wallet_obj.current_balance
    updated_balance = current_balance + amount
    agent_wallet_obj.current_balance = updated_balance
    agent_wallet_obj.save()

In [3]:
def create_transaction_log(user_id, amount, rid, wallet_balance_before, wallet_balance_after):
    transaction_obj = TransactionLog(
      date=datetime.now(),
      transacted_by_id=user_id,
      transacted_via_id=1,
      data_entered_by_id=user_id,
      amount=amount,
      transaction_direction_id=2,  # from agent wallet to aavin
      transaction_mode_id=1,  # Upi
      transaction_status_id=2,  # completed
      transaction_id=rid,
      transaction_approval_status_id=1,  # Accepted
      transaction_approved_by_id=1,
      transaction_approved_time=datetime.now(),
      wallet_balance_before_this_transaction=wallet_balance_before,
      wallet_balance_after_transaction_approval=wallet_balance_after,
      description='Amount for ordering the product from agent wallet',
      modified_by_id=user_id
    )
    transaction_obj.save()

In [4]:
def send_message_via_netfision(mobile, message):
    payload = {'ClientId': 'c12015f4-2ae8-4c9f-bd74-51379169c9e5', 'ApiKey' : '622de6e4-91da-4e3b-9fb1-2262df7baff8', 'SenderID' : 'AAVINM', 'fl':'0', 'gwid':'2', 'sid':'AAVINM'}
    headers = {}
    url     = 'http://sms.tnvt.in/vendorsms/pushsms.aspx'
    payload['msg'] = message
    payload['msisdn'] = 7010349412, 9003832999
    res = requests.post(url, data=payload, headers=headers)

In [5]:
def check_order_expiry_available(sale_obj):
    indian = pytz.timezone('Asia/Kolkata')
    order_date = str(sale_obj.date)
    order_expiry_date = datetime.strptime(order_date, "%Y-%m-%d") - timedelta(days=sale_obj.route.session.expiry_day_before)
    order_expiry_time = sale_obj.route.order_expiry_time
    order_expiry_date_with_time = datetime.combine(order_expiry_date, order_expiry_time)
    indian_datetime = datetime.now().astimezone(indian).replace(tzinfo=None)
    if indian_datetime < order_expiry_date_with_time:
        return True
    else:
        return False

In [246]:
def execute_enquiry_for_month():
    one_month_before_date = datetime.now() - relativedelta.relativedelta(months=1)   
    payment_request_obj = PaymentRequest.objects.filter(time_created__date__gte=one_month_before_date, time_created__date__lte=datetime.now(), payment_request_for_id__in=[1,2])
    overall_canditate = payment_request_obj.count()
    success_count = 0
    payment_auto_enquiry_obj = PaymentAutoEnquiry(run_time=datetime.now(),
                                             run_interval_start=one_month_before_date,
                                             run_interval_end=datetime.now(),
                                             run_type='Back Log Run',
                                             candidate_count=overall_canditate,
                                             success_count=0)

    payment_auto_enquiry_obj.save()
    for payment_request in payment_request_obj:

        enquiry_status = '111'
        if PaymentRequestResponse.objects.filter(payment_request_id=payment_request.id).exists():
            payment_response_obj = PaymentRequestResponse.objects.get(payment_request_id=payment_request.id)
    #         get the un success requestsresponse
            if not payment_response_obj.status.id == 1:
                enquiry_status = get_enquiry_responce_from_server(payment_request.rid)
        else:
            enquiry_status = get_enquiry_responce_from_server(payment_request.rid)
        if enquiry_status == '000':
            success_count += 1
    #         GET User id for the agent
            user_id = PaymentRequestUserMap.objects.get(payment_request_id=payment_request.id).payment_intitated_by.id
            business_agent_obj = BusinessAgentMap.objects.get(business__user_profile__user_id=user_id)
            agent_id = business_agent_obj.agent.id
            agent_wallet_obj = AgentWallet.objects.get(agent_id=agent_id)
    #         update agent wallet with requested amount
            send_amount_to_agent_wallet(agent_id, payment_request.amt)
    #         make transaction log for the requested amount
            create_transaction_log(user_id, payment_request.amt, payment_request.rid, agent_wallet_obj.current_balance, agent_wallet_obj.current_balance + payment_request.amt)
            requested_time = payment_request.time_created.astimezone(indian).strftime("%d-%m-%Y %I:%M %p")
            message = 'Dear ' + str(business_agent_obj.agent.first_name) + '(' + str(business_agent_obj.business.code) + ') Payment initiated at ' + str(requested_time) + ' has been resolved. The money ' + str(payment_request.amt) +' has been deposited to your wallet. Sorry for the inconvenience.'
    #         send sms to agent abot amount deposite to wallet
            send_message_via_netfision(business_agent_obj.agent.agent_profile.mobile, message)
#             update auto enquriy log
            payment_auto_enquiry_property_obj = PaymentAutoEnquiryProperty(payment_auto_enquiry_id=payment_auto_enquiry_obj.id,
                                                                          payment_request_id=payment_request.id,
                                                                          amount_sent_to_wallet=payment_request.amt,
                                                                          )
            payment_auto_enquiry_property_obj.save()
    payment_auto_enquiry_obj.success_count = success_count
    payment_auto_enquiry_obj.save()
    

In [233]:
def execute_enquiry_script(execute_option):
    if execute_option == 'per_fifteen_minute':
        half_hour_before_time = datetime.now() - timedelta(hours = 0.5)
        half_hour_before_time = half_hour_before_time.strftime('%Y-%m-%d %H:%M:%S')
        payment_request_obj = PaymentRequest.objects.filter(time_created__gte=half_hour_before_time, time_created__lte=datetime.now(), payment_request_for_id__in=[1,2])
        run_type = 'Priority Run'
    elif execute_option == 'per_day':
        payment_request_obj = PaymentRequest.objects.filter(time_created__date=datetime.now(), payment_request_for_id__in=[1,2])
        run_type = 'Regular Run'
    overall_canditate = payment_request_obj.count()
    success_count = 0
    payment_auto_enquiry_obj = PaymentAutoEnquiry(run_time=datetime.now(),
                                             run_interval_start=half_hour_before_time,
                                             run_interval_end=datetime.now(),
                                             run_type=run_type,
                                             candidate_count=overall_canditate,
                                             success_count=0)

    payment_auto_enquiry_obj.save()
# loop through paymenrequest
    for payment_request in payment_request_obj:
        enquiry_status = '111'
        if PaymentRequestResponse.objects.filter(payment_request_id=payment_request.id).exists():
            payment_response_obj = PaymentRequestResponse.objects.get(payment_request_id=payment_request.id)
    #         get the un success requestsresponse
            if not payment_response_obj.status.id == 1:
                enquiry_status = get_enquiry_responce_from_server(payment_request.rid)
        else:
            enquiry_status = get_enquiry_responce_from_server(payment_request.rid)
        if enquiry_status == '000':
            success_count += 1
    #         GET User id for the agent
            user_id = PaymentRequestUserMap.objects.get(payment_request_id=payment_request.id).payment_intitated_by.id
            business_agent_obj = BusinessAgentMap.objects.get(business__user_profile__user_id=user_id)
            agent_id = business_agent_obj.agent.id
            agent_wallet_obj = AgentWallet.objects.get(agent_id=agent_id)
    #         update agent wallet with requested amount
            send_amount_to_agent_wallet(agent_id, payment_request.amt)
    #         make transaction log for the requested amount
            create_transaction_log(user_id, payment_request.amt, payment_request.rid, agent_wallet_obj.current_balance, agent_wallet_obj.current_balance + payment_request.amt)
            requested_time = payment_request.time_created.astimezone(indian).strftime("%d-%m-%Y %I:%M %p")
            message = 'Dear ' + str(business_agent_obj.agent.first_name) + '(' + str(business_agent_obj.business.code) + ') Payment initiated at ' + str(requested_time) + ' has been resolved. The money ' + str(payment_request.amt) +' has been deposited to your wallet. Sorry for the inconvenience.'
    #         send sms to agent abot amount deposite to wallet
            send_message_via_netfision(business_agent_obj.agent.agent_profile.mobile, message)
            payment_auto_enquiry_property_obj = PaymentAutoEnquiryProperty(payment_auto_enquiry_id=payment_auto_enquiry_obj.id,
                                                                          payment_request_id=payment_request.id,
                                                                          amount_sent_to_wallet=payment_request.amt,
                                                                          )
            payment_auto_enquiry_property_obj.save()
            if payment_request.payment_request_for_id == 1:
                temp_sale_group_payment_request_map = SaleGroupPaymentRequestMap.objects.get(payment_request_id=payment_request.id)
                temp_sale_group_ids = list(SaleGroupPaymentRequestMap.objects.filter(payment_request_id=payment_request.id).values_list('temp_sale_group', flat=True))
                business_id = TempSaleGroup.objects.get(id=temp_sale_group_ids[0]).business_id
                business_obj = BusinessAgentMap.objects.get(business_id=business_id).business
                agent_obj = BusinessAgentMap.objects.get(business_id=business_id).agent
                agent_wallet_obj = AgentWallet.objects.get(agent_id=agent_obj.id)
    #             loop thorough the tempsale groups
                for temp_sale_group_id in temp_sale_group_ids:
                    temp_sale_group_data_obj = TempSaleGroup.objects.get(id=temp_sale_group_id)
                    route_trace_indent_status_id = RouteTrace.objects.get(date=temp_sale_group_data_obj.date, route_id=temp_sale_group_data_obj.route.id).indent_status.id
    #                 check indent closed or not
                    if route_trace_indent_status_id == 1:
    #                     check id order time closed or not
                        if check_order_expiry_available(temp_sale_group_data_obj):

                            temp_sale_group_obj = TempSaleGroup.objects.filter(id=temp_sale_group_id).values()[0]

                            if not SaleGroup.objects.filter(date=temp_sale_group_obj['date'], session_id=temp_sale_group_obj['session_id'], business_id=business_id).exists():
                                temp_sale_group_obj['id'] = None
                                main_sale_group_obj = SaleGroup.objects.create(**temp_sale_group_obj)
                                vehicle_obj = RouteVehicleMap.objects.get(route_id=main_sale_group_obj.route.id).vehicle
                                # sale
                                temp_sale_ids = list(TempSale.objects.filter(temp_sale_group_id=temp_sale_group_id).values_list('id', flat=True))
                                for temp_sale_id in temp_sale_ids:
                                    temp_sale_obj = TempSale.objects.filter(id=temp_sale_id).values()[0]
                                    temp_sale_obj['id'] = None
                                    temp_sale_obj['sale_group_id'] = main_sale_group_obj.id
                                    temp_sale_obj.pop('temp_sale_group_id', None)
                                    main_sale_obj = Sale.objects.create(**temp_sale_obj)

                                    # add sale to route trace
                                    if RouteTrace.objects.filter(date=main_sale_group_obj.date, session_id=main_sale_group_obj.session_id, route_id=main_sale_group_obj.route.id).exists():
                                        route_trace_obj = RouteTrace.objects.get(date=main_sale_group_obj.date, session_id=main_sale_group_obj.session_id, route_id=main_sale_group_obj.route.id)
                                        if RouteTraceWiseSaleSummary.objects.filter(route_trace_id=route_trace_obj.id, product_id=main_sale_obj.product.id).exists():
                                            route_trace_sale_summary_obj = RouteTraceWiseSaleSummary.objects.get(route_trace_id=route_trace_obj.id, product_id=main_sale_obj.product.id)
                                            route_trace_sale_summary_obj.quantity += Decimal(main_sale_obj.count)
                                            route_trace_sale_summary_obj.save()
                                        else:
                                            route_trace_sale_summary_obj = RouteTraceWiseSaleSummary(route_trace_id=route_trace_obj.id, product_id=main_sale_obj.product.id, quantity= main_sale_obj.count)
                                            route_trace_sale_summary_obj.save()
                                    else:
                                        route_trace_obj = RouteTrace(indent_status_id=1, #initiated
                                                                      route_id=main_sale_group_obj.route.id,
                                                                      vehicle_id=vehicle_obj.id,
                                                                      date=main_sale_group_obj.date,
                                                                      session_id=main_sale_group_obj.session_id,
                                                                      driver_name=vehicle_obj.driver_name,
                                                                      driver_phone=vehicle_obj.driver_mobile)
                                        route_trace_obj.save()
                                        route_trace_sale_summary_obj = RouteTraceWiseSaleSummary(route_trace_id=route_trace_obj.id,
                                                                                                  product_id=main_sale_obj.product.id,
                                                                                                  quantity=main_sale_obj.count)
                                        route_trace_sale_summary_obj.save()
                                create_transaction_log(business_obj.user_profile.user.id, main_sale_group_obj.total_cost, payment_request.rid, agent_wallet_obj.current_balance, agent_wallet_obj.current_balance - main_sale_group_obj.total_cost)
                                agent_wallet_obj.current_balance -= main_sale_group_obj.total_cost
                                agent_wallet_obj.save()
                                message = 'Dear ' + str(business_agent_obj.agent.first_name) + '(' + str(business_agent_obj.business.code) + ') Your ' + str(main_sale_group_obj.session.name) + ' Order has been placed using wallet amout. Current wallet balance is ' + str(agent_wallet_obj.current_balance)
                        #         send sms to agent abot order placed
                                send_message_via_netfision(business_agent_obj.agent.agent_profile.mobile, message)
#                                 update sale detais in auto enquiry log
                                payment_auto_enquiry_property_obj.is_order_placed = True
                                if main_sale_group_obj.session.id == 1:
                                    payment_auto_enquiry_property_obj.morning_sale_group_id = main_sale_group_obj.id
                                    payment_auto_enquiry_property_obj.morning_order_amount = main_sale_group_obj.total_cost
                                else:
                                    payment_auto_enquiry_property_obj.evening_sale_group_id = main_sale_group_obj.id
                                    payment_auto_enquiry_property_obj.evening_order_amount = main_sale_group_obj.total_cost
                                payment_auto_enquiry_property_obj.save()
                                temp_sale_group_payment_request_map.sale_group.add(main_sale_group_obj.id)
                            #   add sale to online counter
                                employee_id = 3
                                if CounterEmployeeTraceMap.objects.filter(employee_id=employee_id, is_active=True,
                                                                                              collection_date=datetime.now()).exists():
                                    counter_employee_trace_obj = CounterEmployeeTraceMap.objects.filter(employee_id=employee_id, is_active=True,
                                                                            collection_date=datetime.now())[0]
                                else:
                                    counter_employee_trace_obj = CounterEmployeeTraceMap(counter_id=23, employee_id=employee_id,collection_date=datetime.now(),
                                                                                          start_date_time=datetime.now())
                                    counter_employee_trace_obj.save()
                                if CounterEmployeeTraceSaleGroupMap.objects.filter(
                                        counter_employee_trace_id=counter_employee_trace_obj.id).exists():
                                    counter_sale_group_obj = CounterEmployeeTraceSaleGroupMap.objects.get(
                                        counter_employee_trace_id=counter_employee_trace_obj.id)
                                    counter_sale_group_obj.sale_group.add(main_sale_group_obj.id)
                                    counter_sale_group_obj.save()
                                else:
                                    counter_sale_group_obj = CounterEmployeeTraceSaleGroupMap(
                                        counter_employee_trace_id=counter_employee_trace_obj.id)
                                    counter_sale_group_obj.save()
                                    counter_sale_group_obj.sale_group.add(main_sale_group_obj.id)
                                    counter_sale_group_obj.save()

                                temp_sale_group_payment_request_map.save()
                                # update transactino log status
                                try:
                                    transaction_log_obj = TransactionLog.objects.filter(transaction_id=payment_request.rid)[0]
                                    transaction_log_obj.transaction_status_id = 2
                                    transaction_log_obj.save()
                                except Exception as e:
                                    print(e)
    payment_auto_enquiry_obj.success_count = success_count
    payment_auto_enquiry_obj.save()
    

In [247]:
from datetime import datetime, timedelta
import pytz
from Crypto.Cipher import AES
import hashlib
import os
import random
from base64 import b64decode, b64encode
from decimal import Decimal
from dateutil import relativedelta
pad_text_dict = {
    0: '\x00',
    1: '\x01',
    2: '\x02',
    3: '\x03',
    4: '\x04',
    5: '\x05',
    6: '\x06',
    7: '\x07',
    8: '\x08',
    9: '\x09',
    10: '\x0a',
    11: '\x0b',
    12: '\x0c',
    13: '\x0d',
    14: '\x0e',
    15: '\x0f',
    16: '\x10'
    
}
import requests
encryption_key = '0!EaW^9FwiZSlWF7'
checksum_key = '^aR^'
# execute_enquiry_script('per_fifteen_minute')
# execute_enquiry_script('per_day')
# execute_enquiry_for_month
                        
        
            

In [199]:
datetime.now().astimezone(indian).replace(tzinfo=None)

datetime.datetime(2020, 8, 20, 16, 20, 38, 736692)

In [38]:
datetime.now().strftime('%H:%M:%S')

'06:29:31'

In [253]:
half_hour_before_time = datetime.now().astimezone(indian).replace(tzinfo=None)
half_hour_before_time.strftime('%c')

'Thu Aug 20 18:42:36 2020'

In [45]:
half_hour_before_time = datetime.now() - timedelta(hours = 0.5)
half_hour_before_time = half_hour_before_time.strftime('%Y-%m-%d %H:%M:%S')
for payment in PaymentRequest.objects.filter(time_created__gte=half_hour_before_time, time_created__lte=datetime.now()):
    print(payment.time_created)



/Users/gramasamy/Software/virtualenvs/aavin/lib/python3.6/site-packages/django/db/models/fields/__init__.py:1451: RuntimeWarning: DateTimeField PaymentRequest.time_created received a naive datetime (2020-08-20 06:03:05) while time zone support is active.
  RuntimeWarning)
/Users/gramasamy/Software/virtualenvs/aavin/lib/python3.6/site-packages/django/db/models/fields/__init__.py:1451: RuntimeWarning: DateTimeField PaymentRequest.time_created received a naive datetime (2020-08-20 06:33:05.175794) while time zone support is active.
  RuntimeWarning)


0

In [107]:
def check():
    import pytz
    indian = pytz.timezone('Asia/Kolkata')
    temp_obj = TempSaleGroup.objects.get(id=1380)
    order_date = str(temp_obj.date)
    print(order_date)
    order_expiry_date = datetime.strptime(order_date, "%Y-%m-%d") - timedelta(days=temp_obj.route.session.expiry_day_before)
    order_expiry_time = temp_obj.route.order_expiry_time
    order_expiry_date_with_time = datetime.combine(order_expiry_date, order_expiry_time)
    indian_datetime = datetime.now().astimezone(indian).replace(tzinfo=None)
    if indian_datetime < order_expiry_date_with_time:
        return True
    else:
        return False

In [163]:
temp_sale_group_obj = TempSaleGroup.objects.get(id=1380)
date = temp_sale_group_obj.time_created.astimezone(indian).strftime("%d-%m-%Y %I:%M %p")
print(date)
datetime.strptime(date[0], "%H:%M:%S")


20-08-2020 07:38 AM


ValueError: time data '2' does not match format '%H:%M:%S'

In [166]:
user_id = 2526
business_agent_obj = BusinessAgentMap.objects.get(business__user_profile__user_id=user_id)
requested_time = payment_request.time_created.astimezone(indian).strftime("%d-%m-%Y %I:%M %p")
message = 'Dear ' + str(business_agent_obj.agent.first_name) + '(' + str(business_agent_obj.business.code) + ') Payment initiated at ' + str(requested_time) + ' has been resolved. The money ' + str(payment_request.amt) +' has been deposited to your wallet. Sorry for the inconvenience.'
send_message_via_netfision('7010349412', message)

In [223]:
SaleGroup.objects.filter(date='2020-08-21', business_id=2243).delete()

(6,
 {'main.TempSaleForEdit': 0,
  'main.CounterEmployeeTraceSaleGroupMap_sale_group': 0,
  'main.SaleGroupPaymentRequestMap_sale_group': 1,
  'main.SaleGroupEditPaymentRequestMap_sale_group': 0,
  'main.Sale': 3,
  'main.SaleGroup': 2})

In [244]:
from dateutil import relativedelta
one_month_before_date = datetime.now() - relativedelta.relativedelta(months=1)   
payment_request_obj = PaymentRequest.objects.filter(time_created__date__gte=one_month_before_date, time_created__date__lte=datetime.now(), payment_request_for_id__in=[1,2])
payment_request_obj.count()

1049

In [6]:
for payment_request in PaymentRequest.objects.all():
    if EnquiryRequestResponse.objects.filter(rid=payment_request.rid).exists():
        enquiry_response_obj = EnquiryRequestResponse.objects.filter(rid=payment_request.rid).order_by('-time_created')[0]
        payment_request.enquiry_response_status_latest_id = enquiry_response_obj.status.id
        payment_request.save()
        print('updated from enquiry log')
    elif PaymentRequestResponse.objects.filter(payment_request_id=payment_request.id).exists():
        payment_response_pbj = PaymentRequestResponse.objects.get(payment_request_id=payment_request.id)
        payment_request.enquiry_response_status_latest_id = payment_response_pbj.status.id
        payment_request.save()
        print('updated from responce log')
        
        
        
        

updated from responce log
updated from responce log
updated from responce log
updated from enquiry log
updated from responce log
updated from responce log
updated from enquiry log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from enquiry log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from enquiry log
updated from enquiry log
updated from enquiry log
updated from responce log
updated from enquiry log
updated from enquiry log
updated from responce log
updated from responce log
updated from enquiry log
updated from responce log
updated from enquiry log
updated from responce log
updated from responce log
updated from responce log
updated from enquiry log
updated from enquiry log
updated from enquiry log
updated from responce log
updated from responce log
updated from enquiry log


updated from responce log
updated from enquiry log
updated from enquiry log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from enquiry log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from en

updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from enquiry log
updated from enquiry log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from responce log
updated from r

In [41]:
import datetime
from datetime import timedelta
half_hour_before_time = datetime.datetime.now() - timedelta(minutes = 10)
half_hour_before_time = half_hour_before_time.strftime('%Y-%m-%d %H:%M:%S')
half_hour_before_time

'2020-08-21 12:28:52'

In [53]:
def get_payment_request_user_first_name(payment_request_id):
    user_id = PaymentRequestUserMap.objects.get(payment_request_id=payment_request_id).payment_intitated_by.id
    business_agent_obj = BusinessAgentMap.objects.get(business__user_profile__user_id=user_id)
    return str(business_agent_obj.business.user_profile.user.first_name) 

def get_payment_request_user_booth_code(payment_request_id):
    user_id = PaymentRequestUserMap.objects.get(payment_request_id=payment_request_id).payment_intitated_by.id
    business_agent_obj = BusinessAgentMap.objects.get(business__user_profile__user_id=user_id)
    return str(business_agent_obj.business.code)


In [54]:
import pandas as pd
from_date = '2020-08-20'
to_date = '2020-08-21'
payment_auto_enquiry_obj = PaymentAutoEnquiry.objects.filter(run_time__date__gte=from_date, run_time__date__lte=to_date)
payment_auto_enquiry_list = list(payment_auto_enquiry_obj.values_list('id', 'run_time', 'run_interval_start', 'run_interval_end', 'run_type', 'candidate_count', 'success_count')) 
payment_auto_enquiry_column = ['id', 'run_time', 'start_time', 'end_time', 'run_type', 'candidate_count', 'success_count']
payment_auto_enquiry_df = pd.DataFrame(payment_auto_enquiry_list, columns=payment_auto_enquiry_column)
payment_auto_enquiry_data = payment_auto_enquiry_df.to_dict('r')

In [56]:
payment_auto_enquiry_ids = list(PaymentAutoEnquiry.objects.filter(run_time__date__gte=from_date, run_time__date__lte=to_date).values_list('id', flat=True))

payment_auto_enquiry_property_obj = PaymentAutoEnquiryProperty.objects.filter(payment_auto_enquiry_id__in=payment_auto_enquiry_ids)
payment_auto_enquiry_property_list = list(payment_auto_enquiry_property_obj.values_list('id', 'payment_auto_enquiry', 'payment_request', 'amount_sent_to_wallet', 'is_order_placed', 'morning_sale_group', 'morning_order_amount', 'evening_sale_group', 'evening_order_amount'))
payment_auto_enquiry_property_column = ['id', 'payment_auto_enquiry', 'payment_request', 'amount_sent_to_wallet', 'is_order_placed', 'morning_sale_group', 'morning_order_amount', 'evening_sale_group', 'evening_order_amount']
payment_auto_enquiry_property_df = pd.DataFrame(payment_auto_enquiry_property_list, columns=payment_auto_enquiry_property_column)
payment_auto_enquiry_property_df['first_name'] = payment_auto_enquiry_property_df.apply(lambda x: get_payment_request_user_first_name(x['payment_request']), axis=1)
payment_auto_enquiry_property_df['booth_code'] = payment_auto_enquiry_property_df.apply(lambda x: get_payment_request_user_booth_code(x['payment_request']), axis=1)

payment_auto_enquiry_property_df.groupby('payment_auto_enquiry').apply(lambda x: x.to_dict('r')).to_dict()




{2: [{'id': 1,
   'payment_auto_enquiry': 2,
   'payment_request': 1245,
   'amount_sent_to_wallet': Decimal('2764.00'),
   'is_order_placed': False,
   'morning_sale_group': None,
   'morning_order_amount': Decimal('0.00'),
   'evening_sale_group': None,
   'evening_order_amount': Decimal('0.00'),
   'first_name': 'm.ganesh kumar',
   'booth_code': '2649'}]}

In [3]:
from django.shortcuts import render
# authendication
from rest_framework.decorators import api_view, permission_classes
from django.contrib.auth import authenticate, logout, login
from rest_framework.authtoken.models import Token
from rest_framework.permissions import AllowAny
from django.contrib.auth.hashers import make_password
# Create your views here.
from django.db.models import Max
from collections import defaultdict, OrderedDict, Counter
# creating views for REST / function based views
from rest_framework import status
from rest_framework.response import Response
from django.contrib.auth import authenticate, logout, login
from django.contrib.auth.models import User
import random
import plivo
from main.models import *
from report.models import *
import pandas as pd
from decimal import Decimal
from datetime import timedelta, date
# from datetime import datetime
import datetime
import dateutil.relativedelta
from base64 import b64encode, b64decode
#  Plivo credentials
# import plivo
from random import randint
from base64 import b64decode, b64encode
from django.core.files.base import ContentFile
from django.db.models import Sum, Max
from pytz import timezone
from django.db.models import Sum
from calendar import monthrange, month_name
import pytz
indian = pytz.timezone('Asia/Kolkata')

# canvas
import os
from reportlab.pdfgen import canvas
from reportlab.platypus import BaseDocTemplate, SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.colors import HexColor
from main.temp_pdf_generate import generate_pdf_for_merging_temp_with_main
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from num2words import num2words

In [4]:
def boothwise_customer_card_details(booth_code,month,year):
    data_dict = {}
    icustomer_sale_group_obj = ICustomerSaleGroup.objects.filter(date__year=year,date__month=month,business__code=booth_code)
    for sale_group in icustomer_sale_group_obj:
        if not sale_group.icustomer_id in data_dict:
            data_dict[sale_group.icustomer_id] = {
                "card_no" : ICustomerMonthlyOrderTransaction.objects.get(icustomer_id=sale_group.icustomer_id).milk_card_number,
                "sl_no": ICustomerSerialNumberMap.objects.get(month=month,year=year,icustomer_id=sale_group.icustomer_id).serial_number,
                "customer_name": sale_group.icustomer.user_profile.user.first_name +" "+sale_group.icustomer.user_profile.user.last_name,
                "customer_id":sale_group.icustomer.customer_code,
                "TM 500":0,
                "STD 500":0,
                "STD 250":0,
                "FCM 500":0
            }
        for sale in ICustomerSale.objects.filter(icustomer_sale_group_id=sale_group.id):
            if not sale.product.short_name in data_dict[sale_group.icustomer_id]:
                data_dict[sale_group.icustomer_id][sale.product.short_name] = 0
            data_dict[sale_group.icustomer_id][sale.product.short_name] += sale.count
    data_dict["user_name"] = "sunesh"
    data = boothwise_customer_card_details_pdf_gen(data_dict,month,year,booth_code)
    return data_dict

def boothwise_customer_card_details_pdf_gen(data_dict,month,year,booth_code):
   
    month = month
    year = year
    days = monthrange(year, month)
    datetime_object = datetime.datetime.strptime(str(month), "%m")
    month_name = datetime_object.strftime("%B")
   
    file_name = "boothwise_customer_card_details for ( "+str(month_name)+"-"+str(year)+" )"+ '.pdf'
    file_path = os.path.join('static/media/', file_name)
    mycanvas = canvas.Canvas(file_path, pagesize=A4)
    mycanvas.setLineWidth(0)
   
    mycanvas.setFont('Helvetica', 12.5)
    mycanvas.drawCentredString(300, 820,'The Coimbatore District Co-Operative Milk Producers Union Ltd, Pachapalayam, Coimbatore - 641 010')
    mycanvas.setFont('Helvetica', 11)
    mycanvas.drawString(30, 795, 'Boothwise Customer Card Details')
    mycanvas.drawCentredString(300, 795, str(month_name)+"-"+str(year))
    mycanvas.drawRightString(565, 795, "Booth No : "+str(booth_code))
   
    #heading
    mycanvas.setFont('Helvetica', 12)
    head_y = 765
    head_x = 30
   
    mycanvas.drawString(head_x,head_y,"Sl No")
    mycanvas.drawString(head_x+50,head_y,"Card")
    mycanvas.drawString(head_x+55,head_y-15,"NO")
   
    mycanvas.drawString(head_x+110,head_y,"Card")
    mycanvas.drawString(head_x+110,head_y-15,"Sl No")
   
    mycanvas.drawString(head_x+200,head_y,"Name")
    mycanvas.drawString(head_x+300,head_y,"Customer Id")
   
    mycanvas.drawString(head_x+391,head_y,"TM")
    mycanvas.drawString(head_x+390,head_y-15,"500")
   
    mycanvas.drawString(head_x+425,head_y,"STD")
    mycanvas.drawString(head_x+427,head_y-15,"500")
   
    mycanvas.drawString(head_x+465,head_y,"STD")
    mycanvas.drawString(head_x+467,head_y-15,"250")
   
    mycanvas.drawString(head_x+505,head_y,"FCM")
    mycanvas.drawString(head_x+508,head_y-15,"500")
   
    #heading_top & Bottom Line
   
    mycanvas.line(head_x-5,head_y +15,head_x+535,head_y +15)
    mycanvas.line(head_x-5,head_y -22,head_x+535,head_y -22)
   
    #table_content
    x_axis = 30
    y_axis = 725
    sl_no = 1
    tm500 = 0
    std500 = 0
    std250 = 0
    fcm500 = 0
    for data in data_dict:
        if data != "user_name":
            mycanvas.setFont('Helvetica', 10)
            mycanvas.drawString(x_axis,y_axis,str(sl_no))
            mycanvas.drawString(x_axis+48,y_axis,str(data_dict[data]["card_no"]))
            mycanvas.drawRightString(x_axis+140,y_axis,str(data_dict[data]["sl_no"]))
           
            mycanvas.drawString(x_axis+155,y_axis,str(data_dict[data]["customer_name"])[:21].upper())
            mycanvas.drawString(x_axis+305,y_axis,str(data_dict[data]["customer_id"]))
           
            if data_dict[data]["TM 500"] != 0:
                mycanvas.drawRightString(x_axis+415,y_axis,str(int(data_dict[data]["TM 500"])))
                tm500 += int(data_dict[data]["TM 500"])
            if data_dict[data]["STD 500"] != 0:
                mycanvas.drawRightString(x_axis+450,y_axis,str(int(data_dict[data]["STD 500"])))
                std500 += int(data_dict[data]["STD 500"])
            if data_dict[data]["STD 250"] != 0:
                mycanvas.drawRightString(x_axis+490,y_axis,str(int(data_dict[data]["STD 250"])))
                std250 += int(data_dict[data]["STD 250"])
            if data_dict[data]["FCM 500"] != 0:
                mycanvas.drawRightString(x_axis+530,y_axis,str(int(data_dict[data]["FCM 500"])))
                fcm500 += int(data_dict[data]["FCM 500"])
               
               
            #lines
            mycanvas.line(x_axis-5,y_axis+55,x_axis-5,y_axis-15)
            mycanvas.line(x_axis+33,y_axis+55,x_axis+33,y_axis-15)
            mycanvas.line(x_axis+100,y_axis+55,x_axis+100,y_axis-15)
            mycanvas.line(x_axis+150,y_axis+55,x_axis+150,y_axis-15)
            mycanvas.line(x_axis+290,y_axis+55,x_axis+290,y_axis-15)
            mycanvas.line(x_axis+380,y_axis+55,x_axis+380,y_axis-15)
           
            mycanvas.line(x_axis+420,y_axis+55,x_axis+420,y_axis-15)
            mycanvas.line(x_axis+458,y_axis+55,x_axis+458,y_axis-15)
            mycanvas.line(x_axis+497,y_axis+55,x_axis+497,y_axis-15)
            mycanvas.line(x_axis+535,y_axis+55,x_axis+535,y_axis-15)
           
               
            if sl_no % 34 == 0:
                mycanvas.line(head_x-5,y_axis-15,head_x+535,y_axis-15)
                mycanvas.showPage()
                mycanvas.setFont('Helvetica', 12.5)
                mycanvas.drawCentredString(300, 820,'The Coimbatore District Co-Operative Milk Producers Union Ltd, Pachapalayam, Coimbatore - 641 010')
                mycanvas.setFont('Helvetica', 11)
                mycanvas.drawString(30, 795, 'Boothwise Customer Card Details')
                mycanvas.drawCentredString(300, 795, str(month_name)+"-"+str(year))
                mycanvas.drawRightString(565, 795, "Booth No : "+str(booth_code))

                #heading
                mycanvas.setFont('Helvetica', 12)
                head_y = 765
                head_x = 30

                mycanvas.drawString(head_x,head_y,"Sl No")
                mycanvas.drawString(head_x+50,head_y,"Card")
                mycanvas.drawString(head_x+55,head_y-15,"NO")

                mycanvas.drawString(head_x+110,head_y,"Card")
                mycanvas.drawString(head_x+110,head_y-15,"Sl No")

                mycanvas.drawString(head_x+200,head_y,"Name")
                mycanvas.drawString(head_x+300,head_y,"Customer Id")

                mycanvas.drawString(head_x+391,head_y,"TM")
                mycanvas.drawString(head_x+390,head_y-15,"500")

                mycanvas.drawString(head_x+425,head_y,"STD")
                mycanvas.drawString(head_x+427,head_y-15,"500")

                mycanvas.drawString(head_x+465,head_y,"STD")
                mycanvas.drawString(head_x+467,head_y-15,"250")

                mycanvas.drawString(head_x+505,head_y,"FCM")
                mycanvas.drawString(head_x+508,head_y-15,"500")

                #heading_top & Bottom Line

                mycanvas.line(head_x-5,head_y +15,head_x+535,head_y +15)
                mycanvas.line(head_x-5,head_y -22,head_x+535,head_y -22)

                #table_content
                x_axis = 30
                y_axis = 745
               
        y_axis -= 20
        sl_no += 1
    mycanvas.line(head_x-5,y_axis+25,head_x+535,y_axis+25)
    mycanvas.line(head_x+380,y_axis+5,head_x+535,y_axis+5)
    mycanvas.line(x_axis+380,y_axis+55,x_axis+380,y_axis+5)
    mycanvas.line(x_axis+420,y_axis+55,x_axis+420,y_axis+5)
    mycanvas.line(x_axis+458,y_axis+55,x_axis+458,y_axis+5)
    mycanvas.line(x_axis+497,y_axis+55,x_axis+497,y_axis+5)
    mycanvas.line(x_axis+535,y_axis+55,x_axis+535,y_axis+5)
   
    mycanvas.drawRightString(x_axis+365,y_axis+10,"G R A N D   T O T A L")
   
    mycanvas.drawRightString(x_axis+415,y_axis+10,str(tm500))
    mycanvas.drawRightString(x_axis+450,y_axis+10,str(std500))
    mycanvas.drawRightString(x_axis+490,y_axis+10,str(std250))
    mycanvas.drawRightString(x_axis+530,y_axis+10,str(fcm500))
   
    mycanvas.setFont('Times-Italic', 10)      
    indian = pytz.timezone('Asia/Kolkata')          
    mycanvas.drawRightString(580, 10,'Report Generated by: ' + str(data_dict["user_name"] + ", @" + str(datetime.datetime.now().astimezone(indian).strftime("%Y-%m-%d %I:%M:%S"))))
   
    mycanvas.save()


In [5]:
boothwise_customer_card_details(1, 9, 2020)

{4322: {'card_no': '000077',
  'sl_no': 1,
  'customer_name': 'Deivasigamani D',
  'customer_id': 'CBE104158',
  'TM 500': 1.0,
  'STD 500': 0,
  'STD 250': 0,
  'FCM 500': 0},
 4306: {'card_no': '000079',
  'sl_no': 2,
  'customer_name': 'Kandasamy S',
  'customer_id': 'CBE104108',
  'TM 500': 0,
  'STD 500': 1.0,
  'STD 250': 0,
  'FCM 500': 0},
 5345: {'card_no': '000083',
  'sl_no': 3,
  'customer_name': 'Kandhaswamy R',
  'customer_id': 'CBE110197',
  'TM 500': 0,
  'STD 500': 1.0,
  'STD 250': 0,
  'FCM 500': 0},
 4751: {'card_no': '000119',
  'sl_no': 4,
  'customer_name': 'Ramadurai C.N',
  'customer_id': 'CBE105875',
  'TM 500': 3.0,
  'STD 500': 0,
  'STD 250': 0,
  'FCM 500': 0},
 3924: {'card_no': '000173',
  'sl_no': 5,
  'customer_name': 'Venkata Krishnan L',
  'customer_id': 'CBE102771',
  'TM 500': 0,
  'STD 500': 4.0,
  'STD 250': 0,
  'FCM 500': 0},
 4366: {'card_no': '000208',
  'sl_no': 6,
  'customer_name': 'Malathi Gopalakrishnan ',
  'customer_id': 'CBE100427',
 